# Data Wrangling Project: Cryptocurrency Analysis (BTC & ETH)

## Research Question
**How have the prices of Bitcoin (BTC) and Ethereum (ETH) correlated over the last 5 years, and what are their relative trading volumes?**

This project analyzes the daily price and volume data of the two largest cryptocurrencies to understand their market relationship.

## 1. Gather data

In this section, we will gather data from two sources:
1.  **Dataset 1**: Bitcoin (BTC) historical data manually downloaded from Yahoo Finance (CSV).
2.  **Dataset 2**: Ethereum (ETH) historical data downloaded programmatically using the CoinGecko API.

### Dataset 1: Bitcoin (BTC)

**Source**: Yahoo Finance
**Method**: Manual Download

**Instructions:**
1.  Go to the Yahoo Finance page for Bitcoin: [https://finance.yahoo.com/quote/BTC-USD/history](https://finance.yahoo.com/quote/BTC-USD/history)
2.  Set the **Time Period** to **5 Years** (or the max available if less).
3.  Set the **Frequency** to **Daily**.
4.  Click **Apply**.
5.  Click **Download** to save the CSV file.
6.  Rename the downloaded file to `btc_usd.csv`.
7.  Create a folder named `Dataset` in your project directory (if it doesn't exist) and move the file there: `Dataset/btc_usd.csv`.

*Note: The file has been pre-downloaded to the `Dataset` folder for this analysis.*

In [ ]:
import pandas as pd
import os

# Ensure Dataset directory exists
if not os.path.exists('Dataset'):
    os.makedirs('Dataset')

# Load Dataset 1 (Bitcoin CSV)
btc_path = 'Dataset/btc_usd.csv'

if os.path.exists(btc_path):
    df_btc = pd.read_csv(btc_path)
    print(f"Dataset 1 loaded successfully. Shape: {df_btc.shape}")
    display(df_btc.head())
else:
    print(f"File not found: {btc_path}. Please follow the manual download instructions above.")

### Dataset 2: Ethereum (ETH)

**Source**: Binance API (Alternative Source)
**Method**: Programmatic Download

We fetched the last 5 years of daily market data for Ethereum using the Binance API to ensure data consistency and volume.

In [ ]:
import pandas as pd
import os

# Load Dataset 2 (Ethereum CSV)
eth_path = 'Dataset/eth_usd.csv'

if os.path.exists(eth_path):
    df_eth = pd.read_csv(eth_path)
    print(f"Dataset 2 loaded successfully. Shape: {df_eth.shape}")
    display(df_eth.head())
else:
    print(f"File not found: {eth_path}. Please ensure the data gathering script has run.")

## 2. Assess data

Assess the data according to data quality and tidiness metrics.

List **two** data quality issues and **two** tidiness issues. Assess each data issue visually **and** programmatically, then briefly describe the issue you find. **Make sure you include justifications for the methods you use for the assessment.**

### Quality Issue 1: Data Type Inconsistency (Date Column)

In [ ]:
# Inspecting the dataframe visually
print("Visual Inspection of Date column:")
display(df_btc[['Date']].head())
display(df_eth[['Date']].head())

In [ ]:
# Inspecting the dataframe programmatically
print("BTC Data Types:")
print(df_btc.dtypes)
print("\nETH Data Types:")
print(df_eth.dtypes)

Issue and justification: The `Date` column in both datasets is loaded as `object` (string) instead of `datetime` objects. For time-series analysis and merging, we need them to be proper datetime objects.

### Quality Issue 2: Missing Values

In [ ]:
# Inspecting the dataframe visually
# Visual inspection might not reveal missing values in large datasets, so we rely more on programmatic checks.
display(df_btc.info())
display(df_eth.info())

In [ ]:
# Inspecting the dataframe programmatically
print("Missing values in BTC dataset:")
print(df_btc.isnull().sum())
print("\nMissing values in ETH dataset:")
print(df_eth.isnull().sum())

Issue and justification: We need to ensure there are no missing values (NaN) in critical columns like `Close` or `Volume`. Even if the count is 0, this assessment is necessary to confirm data integrity.

### Tidiness Issue 1: Redundant Columns

In [ ]:
# Inspecting the dataframe visually
display(df_btc.head(2))
display(df_eth.head(2))

In [ ]:
# Inspecting the dataframe programmatically
print("Columns in BTC:", df_btc.columns.tolist())
print("Columns in ETH:", df_eth.columns.tolist())

Issue and justification: The `Adj Close` column is redundant with `Close` for cryptocurrency data (they are identical). We can remove `Adj Close` to tidy the dataset.

### Tidiness Issue 2: Datasets are Separate (Need Merging)

In [ ]:
# Inspecting the dataframe visually
print("We currently have two separate dataframes:")
print("df_btc shape:", df_btc.shape)
print("df_eth shape:", df_eth.shape)

In [ ]:
# Inspecting the dataframe programmatically
# Check if Date ranges overlap for merging
print("BTC Date Range:", df_btc['Date'].min(), "to", df_btc['Date'].max())
print("ETH Date Range:", df_eth['Date'].min(), "to", df_eth['Date'].max())

Issue and justification: The observational unit is "a day in the cryptocurrency market". Currently, the data is split into two tables (BTC and ETH). To compare them directly (e.g., correlation, relative volume), they should be in a single table with columns like `BTC_Price`, `ETH_Price`, etc.

## 3. Clean data

Clean the data to solve the 4 issues corresponding to data quality and tidiness found in the assessing step. **Make sure you include justifications for your cleaning decisions.**

After the cleaning for each issue, please use **either** the visually or programatical method to validate the cleaning was succesful.

At this stage, you are also expected to remove variables that are unnecessary for your analysis and combine your datasets. Depending on your datasets, you may choose to perform variable combination and elimination before or after the cleaning stage. Your dataset must have **at least** 4 variables after combining the data.

In [ ]:
# Clean Issue 1: Convert Date to datetime
# Create copies to avoid SettingWithCopy warnings and preserve original data
df_btc_clean = df_btc.copy()
df_eth_clean = df_eth.copy()

df_btc_clean['Date'] = pd.to_datetime(df_btc_clean['Date'])
df_eth_clean['Date'] = pd.to_datetime(df_eth_clean['Date'])

# Validate
print("BTC Date Type:", df_btc_clean['Date'].dtype)
print("ETH Date Type:", df_eth_clean['Date'].dtype)

In [ ]:
# Clean Issue 2: Handle Missing Values
# Check for missing values before dropping
print("Missing before:", df_btc_clean.isnull().sum().sum(), df_eth_clean.isnull().sum().sum())

df_btc_clean.dropna(inplace=True)
df_eth_clean.dropna(inplace=True)

# Validate
print("Missing after:", df_btc_clean.isnull().sum().sum(), df_eth_clean.isnull().sum().sum())

In [ ]:
# Remove unnecessary variables and combine datasets
# 1. Remove 'Adj Close' as identified in Tidiness Issue 1
if 'Adj Close' in df_btc_clean.columns:
    df_btc_clean.drop(columns=['Adj Close'], inplace=True)
if 'Adj Close' in df_eth_clean.columns:
    df_eth_clean.drop(columns=['Adj Close'], inplace=True)

# 2. Select only Price and Volume for analysis (dropping Open, High, Low)
df_btc_clean = df_btc_clean[['Date', 'Close', 'Volume']]
df_eth_clean = df_eth_clean[['Date', 'Close', 'Volume']]

# 3. Rename columns to distinguish BTC and ETH
df_btc_clean = df_btc_clean.rename(columns={'Close': 'Price_BTC', 'Volume': 'Volume_BTC'})
df_eth_clean = df_eth_clean.rename(columns={'Close': 'Price_ETH', 'Volume': 'Volume_ETH'})

# 4. Combine datasets on Date
df_combined = pd.merge(df_btc_clean, df_eth_clean, on='Date', how='inner')

# Validate
print("Combined Data Shape:", df_combined.shape)
display(df_combined.head())

## 4. Update your data store

Update your local database/data store with the cleaned data, following best practices for storing your cleaned data:

1.  Must maintain different instances of the data (raw and cleaned data)
2.  Resourceful naming (e.g. `combined_dataset.csv`)
3.  Compare the quantity of raw and cleaned data

In [ ]:
# Save the final cleaned dataframe
output_path = 'Dataset/crypto_combined.csv'
df_combined.to_csv(output_path, index=False)

print(f"Cleaned data saved to {output_path}")
print(f"Raw BTC count: {len(df_btc)}, Raw ETH count: {len(df_eth)}")
print(f"Combined Cleaned count: {len(df_combined)}")

## 5. Answer the research question

**4.1:** Define and answer the research question.

**Question:** How have the prices of Bitcoin (BTC) and Ethereum (ETH) correlated over the last 5 years, and what are their relative trading volumes?

In [ ]:
# Analysis and Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate Correlation
correlation = df_combined['Price_BTC'].corr(df_combined['Price_ETH'])
print(f"Correlation between BTC and ETH Prices: {correlation:.4f}")

# Visualization 1: Dual Axis Price Plot
fig, ax1 = plt.subplots(figsize=(14, 7))

color = 'tab:orange'
ax1.set_xlabel('Date')
ax1.set_ylabel('BTC Price (USD)', color=color)
ax1.plot(df_combined['Date'], df_combined['Price_BTC'], color=color, label='BTC Price')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('ETH Price (USD)', color=color)
ax2.plot(df_combined['Date'], df_combined['Price_ETH'], color=color, label='ETH Price')
ax2.tick_params(axis='y', labelcolor=color)

plt.title(f'BTC vs ETH Price Trends (5 Years) - Correlation: {correlation:.2f}')
fig.tight_layout()
plt.show()

# Visualization 2: Scatter Plot with Trendline
plt.figure(figsize=(10, 6))
sns.regplot(x='Price_BTC', y='Price_ETH', data=df_combined, scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
plt.title('Correlation Scatter Plot: BTC vs ETH Prices')
plt.xlabel('BTC Price (USD)')
plt.ylabel('ETH Price (USD)')
plt.show()

**4.2:** Reflection
If I had more time to complete the project, I would...